In [1]:
# import external libraries
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from typing import *
import random
    
import warnings
warnings.filterwarnings('ignore')

# import local modules
from middlewares.scraper_functs import *

# Selenium configuration
service = ChromeService()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
# driver.implicitly_wait(10)
wait = WebDriverWait(driver, 5)
driver.maximize_window()

# Dictionary Definition

dataDict={
    'Company':[],
    'Company_Web_Site':[],
    'Category':[], 
    'Value':[],
    'Value_Cat':[],
    'Scope':[]
}

JobDict={
 
    'Company_Web_Site':[],
    'Title':[], 
    'job_details':[],
    'Job_Description':[],
    'Scope':[]
}

# Global Variables definition

unique_companies_source:int = None
unique_companies_after:int= None

# Url 
url_base = 'https://www.linkedin.com/jobs/search/?currentJobId=3736552023&geoId=102890719&keywords=esg&location=Netherlands&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true'
# url_base_2 ='https://www.linkedin.com/jobs/search/?currentJobId=3740663292&geoId=102890719&keywords=esg&location=Netherlands&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=25'
# url_base_3 = 'https://www.linkedin.com/jobs/search/?currentJobId=3721902157&geoId=102890719&keywords=esg&location=Netherlands&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=50'
# url_base_4= 'https://www.linkedin.com/jobs/search/?currentJobId=3718700757&geoId=102890719&keywords=esg&location=Netherlands&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=75'

# Definint the driver
driver.get(url_base)
# eliminate PopUp
eliminate_popup(driver,wait)
# login
login(driver)




In [2]:
url_base_2 ='https://www.linkedin.com/jobs/search/?currentJobId=3740663292&geoId=102890719&keywords=esg&location=Netherlands&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=25'
url_base_3 = 'https://www.linkedin.com/jobs/search/?currentJobId=3721902157&geoId=102890719&keywords=esg&location=Netherlands&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=50'
url_base_4= 'https://www.linkedin.com/jobs/search/?currentJobId=3718700757&geoId=102890719&keywords=esg&location=Netherlands&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=75'


In [3]:
driver.get(url_base_4)

In [5]:
jobs_lists=driver.find_elements(By.XPATH,'//li[contains(@class,"ember-view")]')
print(jobs_lists)
time.sleep(2*random.random())
for job in jobs_lists:
    time.sleep(1)
    scrolling(driver,job)
    job.click()

    data_gatter_jobs(driver=driver,url=url_base,data_dict=JobDict)

[<selenium.webdriver.remote.webelement.WebElement (session="0556831e41b7543064f3c0acc8569e03", element="099952A26C68D1112F7129EB935FB547_element_187")>, <selenium.webdriver.remote.webelement.WebElement (session="0556831e41b7543064f3c0acc8569e03", element="099952A26C68D1112F7129EB935FB547_element_163")>, <selenium.webdriver.remote.webelement.WebElement (session="0556831e41b7543064f3c0acc8569e03", element="099952A26C68D1112F7129EB935FB547_element_164")>, <selenium.webdriver.remote.webelement.WebElement (session="0556831e41b7543064f3c0acc8569e03", element="099952A26C68D1112F7129EB935FB547_element_165")>, <selenium.webdriver.remote.webelement.WebElement (session="0556831e41b7543064f3c0acc8569e03", element="099952A26C68D1112F7129EB935FB547_element_166")>]


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=118.0.5993.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7C2AD8EF2+54786]
	(No symbol) [0x00007FF7C2A45612]
	(No symbol) [0x00007FF7C28FA64B]
	(No symbol) [0x00007FF7C28FF116]
	(No symbol) [0x00007FF7C29014AA]
	(No symbol) [0x00007FF7C2975394]
	(No symbol) [0x00007FF7C295BE6A]
	(No symbol) [0x00007FF7C2974D02]
	(No symbol) [0x00007FF7C295BC43]
	(No symbol) [0x00007FF7C2930941]
	(No symbol) [0x00007FF7C2931B84]
	GetHandleVerifier [0x00007FF7C2E27F52+3524194]
	GetHandleVerifier [0x00007FF7C2E7D800+3874576]
	GetHandleVerifier [0x00007FF7C2E75D7F+3843215]
	GetHandleVerifier [0x00007FF7C2B75086+694166]
	(No symbol) [0x00007FF7C2A50A88]
	(No symbol) [0x00007FF7C2A4CA94]
	(No symbol) [0x00007FF7C2A4CBC2]
	(No symbol) [0x00007FF7C2

In [ ]:
try:

    ul_carrusel_len=len(driver.find_elements(By.XPATH,'//li[contains(@class,"artdeco-pagination__indicator artdeco-pagination__indicator--number")]'))
    
    if ul_carrusel_len > 0:
        for i in range(ul_carrusel_len):
            print(f'printing page {i+1} of {ul_carrusel_len}')
            driver.find_element(By.XPATH,f'//li[@data-test-pagination-page-btn="{i+1}"]').click()
            time.sleep(1*random.random())
            jobs_lists=driver.find_elements(By.XPATH,'//li[contains(@class,"ember-view")]')
            print(jobs_lists)
            time.sleep(2*random.random())
            for i in range(len(jobs_lists)):
                time.sleep(2)
                jobs_lists[i].click()

                data_gatter_jobs(driver=driver,url=url_base,data_dict=JobDict)
                
    else:
        data_gatter_jobs(driver=driver,url=url_base,data_dict=JobDict)
    
except (NoSuchElementException) as e:
    

    JobDict['Company_Web_Site'].append(url_base)
    JobDict['Title'].append(None)
    JobDict['job_details'].append(None)
    JobDict['Job_Description'].append(None)
    JobDict['Scope'].append('Netherlands')  

except StaleElementReferenceException:
    print(jobs_lists)

In [6]:
df=pd.DataFrame(JobDict)

In [7]:
df

,Company_Web_Site,Title,job_details,Job_Description,Scope
0,https://www.linkedin.com/jobs/search/?currentJ...,Corporate Counsel ESG,"Madison People International · The Hague, Sout...",About the job\nOur client is a global leader i...,Netherlands
1,https://www.linkedin.com/jobs/search/?currentJ...,Corporate Counsel ESG,"Madison People International · The Hague, Sout...",About the job\nOur client is a global leader i...,Netherlands


In [8]:
df.to_excel('data/ESG_jobs_v2_4.xlsx',index=False)

In [ ]:
df.drop_duplicates(subset=['Title','job_details','Job_Description'],inplace=True)

In [15]:
import os
import pandas as pd

# Specify the folder path containing the Excel files
folder_path = r'C:\Users\Ariel\OneDrive\Programacion\REPOS\LinkedinPlain\data\clean'

# Get a list of all Excel files in the folder
excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx') or file.endswith('.xls')]

# Initialize an empty list to store individual DataFrames
dfs = []

# Read each Excel file into a DataFrame and append it to the list
for file in excel_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path)
    dfs.append(df)

# Concatenate all DataFrames row-wise
result_df = pd.concat(dfs, axis=0, ignore_index=True)

# Print the resulting DataFrame


In [16]:
result_df

,Company_Web_Site,Title,job_details,Job_Description,Scope
0,https://www.linkedin.com/jobs/search/?currentJ...,ESG Internal Control Manager,NaN,About the job,Netherlands
1,https://www.linkedin.com/jobs/search/?currentJ...,ESG/CSRD reporting specialists,"Robert Walters · Rotterdam, South Holland, Net...",About the job\nWe’re currently recruiting for ...,Netherlands
2,https://www.linkedin.com/jobs/search/?currentJ...,"ESG Consultant | 32 - 40 uur per week, Amersfoort","CROP accountants & adviseurs · Amersfoort, Utr...",About the job\nThis job is sourced from a job ...,Netherlands
3,https://www.linkedin.com/jobs/search/?currentJ...,"(Senior) Manager ESG, Rotterdam","Crowe Peak · Rotterdam, South Holland, Netherl...",About the job\nThis job is sourced from a job ...,Netherlands
4,https://www.linkedin.com/jobs/search/?currentJ...,(Senior) Manager ESG,"RAAS · Rotterdam, South Holland, Netherlands ...",About the job\nWat jij gaat doen\n\nIn deze st...,Netherlands
...,...,...,...,...,...
76,https://www.linkedin.com/jobs/search/?currentJ...,Global Head of ESG (f/m/x),Ward Howell International · European Economic ...,About the job\nWard Howell is known as the sec...,Netherlands
77,https://www.linkedin.com/jobs/search/?currentJ...,ESG Business Controller,"Impactsearch - Finance Recruitment · Beesd, Ge...",About the job\nBedrijfsprofiel\nOnze opdrachtg...,Netherlands
78,https://www.linkedin.com/jobs/search/?currentJ...,ESG Business Controller,"Impactsearch - Finance Recruitment · Beesd, Ge...",About the job\nBedrijfsprofiel\nOnze opdrachtg...,Netherlands
79,https://www.linkedin.com/jobs/search/?currentJ...,Corporate Counsel ESG,"Madison People International · The Hague, Sout...",About the job\nOur client is a global leader i...,Netherlands


In [18]:
result_df.drop_duplicates(subset=['Title','job_details','Job_Description'],inplace=True)


In [ ]:
result_df.to_excel('data/ESG_jobs__final.xlsx',index=False)